In [1]:
from typing import List, Union
from __future__ import annotations
from random import shuffle, randint, choice
from IPython.display import clear_output
from time import sleep

In [62]:
class Card:
    def __init__(self, rank: int, suit: int, is_flipped: bool=False):
        self.__rank = rank
        self.__suit = suit
        self.__is_flipped = is_flipped

        if self == 1:
            self.__value = 11
        elif self.rank >= 2 and self.rank <= 10:
            self.__value = self.rank
        else:
            self.__value = 10
    
    def __repr__(self):
        if self.is_flipped:
            return '🂠'
        ranks = ['A',
                 '2',
                 '3',
                 '4',
                 '5',
                 '6',
                 '7',
                 '8',
                 '9',
                 '10',
                 'J',
                 'Q',
                 'K']
        suits = ['♠',
                 '♥',
                 '♦',
                 '♣']
        rank = ranks[self.rank-1] if self.rank is not None else ''
        suit = suits[self.suit-1] if self.suit is not None else ''
        return rank + suit
    
    def __eq__(self, other: Union[Card, int]):
        if isinstance(other, int):
            return self.rank == other
        return self.rank == other.rank
    
    def flip(self) -> None:
        self.__is_flipped = not self.is_flipped
    
    @property
    def rank(self) -> int:
        return self.__rank
    
    @property
    def suit(self) -> int:
        return self.__suit
    
    @property
    def is_flipped(self) -> bool:
        return self.__is_flipped
    
    @property
    def value(self) -> int:
        return self.__value
    
    @value.setter
    def value(self, value: int):
        assert self == 1, 'card must be an ace to change value'
        assert value in (1, 11), 'new value must be one of (11/1)'
        self.__value = value

class Hand:
    def __init__(self, cards: List[Card]):
        self.__cards = cards
        self.__times_split = 0
    
    def __repr__(self):
        return str(self.cards)
    
    def __iter__(self):
        return iter(self.cards)
    
    def __getitem__(self, i):
        return self.cards[i]
    
    def __eq__(self, other: Union[Hand, List[Card]]):
        return len(self) == len(other) and all(card in other for card in self)
    
    def __len__(self):
        return len(self.cards)
    
    def __orient_hand(self):
        while self.is_busted:
            for card in self:
                if card.value == 11:
                    card.value = 1
                    break
            else:
                break
    
    def add(self, card: Card) -> None:
        self.cards.append(card)
        self.__orient_hand()
    
    def add_before(self, card: Card) -> None:
        self.__cards = [card] + self.cards
        self.__orient_hand()
    
    def split(self) -> Hand:
        self.__times_split += 1
        if self[0].value == 1:
            self[0].value = 11
        card = self.cards.pop()
        return Hand([card])
    
    @property
    def cards(self) -> List[Card]:
        return self.__cards
    
    @property
    def times_split(self) -> bool:
        return self.__times_split

    @property
    def display_score(self) -> str:
        if not [card for card in self if not card.is_flipped]:
            return ''
        score = sum(card.value for card in self if not card.is_flipped)
        if score >= 21:
            return str(score)
        score = f'{score}/{score-10}' if any(card.value == 11 for card in self if not card.is_flipped) else str(score)
        if self.has_flipped_cards:
            score += '+'
        return score

    @property
    def score(self) -> int:
        return sum(card.value for card in self)
    
    @property
    def is_busted(self) -> bool:
        return self.score > 21
    
    @property
    def is_21(self) -> bool:
        return self.score == 21
    
    @property
    def has_flipped_cards(self) -> bool:
        return any(card.is_flipped for card in self)
    
    @property
    def can_split(self) -> bool:
        return len(self) == 2 and self[0].value == self[1].value
    
    @property
    def can_split_same(self) -> bool:
        return len(self) == 2 and self[0] == self[1]
    
    @property
    def is_s17(self) -> bool:
        return len(self) == 2 and 1 in self and 6 in self

class Deck:
    def __init__(self, num_decks: int=1, has_stop_card: bool=False):
        self.__num_decks = num_decks
        self.__has_stop_card = has_stop_card

        self.__num_cards = 52 * self.__num_decks

        self.__reset_cards()
    
    def __reset_cards(self):
        self.__cards = [Card(j+1, k+1, is_flipped=True) for k in range(4) for j in range(13) for i in range(self.__num_decks)]
        shuffle(self.cards)
        if self.has_stop_card:
            self.cards.insert(randint(-75, -60), None)
    
    def shuffle(self) -> None:
        print('Shuffling', end='')
        sleep(0.5)
        for i in range(3):
            print('.', end='')
            sleep(0.5)
        print('.', end='\n\n')
        sleep(0.5)
        self.__reset_cards()
    
    def draw_card(self) -> Card:
        if not self.cards:
            print('Out of cards...')
            self.shuffle()
        card = self.cards.pop(0)
        if card is None:
            self.__insert_reached = True
            print('Insert Reached...')
            card = self.cards.pop(0)
        card.flip()
        return card
    
    @property
    def num_decks(self) -> int:
        return self.__num_decks
    
    @property
    def has_stop_card(self) -> bool:
        return self.__has_stop_card
    
    @property
    def num_cards(self) -> int:
        return self.__num_cards
    
    @property
    def cards(self) -> List[Card]:
        return self.__cards
    
    @property
    def cards_left(self) -> int:
        if not self.has_stop_card:
            return len(self.cards)
        return self.cards.index(None)

class Blackjack:
    def __init__(self, starting_money: int=1000, h17: bool=True, can_dd_after_split: bool=True, resplit_to: int=4, natural_after_split: bool=True, can_split_diff_tens: bool=True, one_hit_ace_split: bool=True, num_decks: int=6, has_stop_card: bool=True, min_bet: int=2, max_bet: int=500):
        assert num_decks > 0, 'must use at least 1 deck'
        assert not (num_decks == 1 and has_stop_card), 'must have at least 2 decks to use stop card'
        assert starting_money >= 0, 'starting money must be a valid number'
        assert min_bet >= 0, 'minimum bet must be a valid number'
        assert max_bet >= min_bet, 'maximum bet must be greater than minimum bet'

        self.__money_left = starting_money
        self.__h17 = h17
        self.__can_dd_after_split = can_dd_after_split
        self.__resplit_to = resplit_to
        self.__natural_after_split = natural_after_split
        self.__can_split_diff_tens = can_split_diff_tens
        self.__one_hit_ace_split = one_hit_ace_split
        self.__deck = Deck(num_decks=num_decks, has_stop_card=has_stop_card)
        self.__min_bet = min_bet
        self.__max_bet = max_bet

        self.__rounds_played = 0
        self.__insert_reached = False
    
    def __refresh_output(self):
        clear_output(wait=True)
        print(f'Round {self.rounds_played+1}')
        print(f'${self.money_left} left', end='\n\n')
    
    def __start_round(self):
        assert self.money_left >= self.min_bet, 'money left must be at least the minimum bet'
        self.__refresh_output()
        if self.money_left >= self.min_bet * 2:
            num_starting_hands = int(input(f'How many hands? (1 -> {min(self.money_left//self.min_bet, 7)}): '))
            assert num_starting_hands >= 1 and num_starting_hands <=7, 'number of player hands must be a valid number'
            assert self.money_left >= self.min_bet * num_starting_hands, 'money left must be at least the minimum bet times number of starting hands'
        else:
            num_starting_hands = 1
        self.__num_starting_hands = num_starting_hands
        self.__player_hands = [Hand([]) for i in range(self.__num_starting_hands)]
        self.__house_hand = Hand([])
    
    def __collect_bets(self):
        self.__bets = []
        for i in range(self.__num_starting_hands):
            self.__refresh_output()
            bet = int(input(f'<Hand {i+1}> Bet (${self.min_bet} -> ${min(self.max_bet, self.money_left - sum(self.__bets))}): '))
            assert bet >= self.min_bet and bet <= min(self.max_bet, self.money_left - sum(self.__bets)), 'bet must be a valid number'
            self.__bets.append(bet)
        print()
    
    def __show_hands(self):
        self.__refresh_output()
        for i in range(len(self.__player_hands)):
            print(f'<Hand {i+1}> ({self.__player_hands[i].display_score}): {self.__player_hands[i]} -> ${self.__bets[i]}')
        print()
        print(f'<House> ({self.__house_hand.display_score}): {self.__house_hand}', end='\n\n')
        sleep(0.5)

    def __deal(self):
        self.__refresh_output()
        if self.__insert_reached:
            self.deck.shuffle()
            self.__insert_reached = False
        self.__show_hands()
        for i in range(len(self.__player_hands)):
            card = self.__deck.draw_card()
            self.__player_hands[i].add(card)
            self.__show_hands()
        card = self.__deck.draw_card()
        card.flip()
        self.__house_hand.add(card)
        self.__show_hands()
        for i in range(len(self.__player_hands)):
            card = self.__deck.draw_card()
            self.__player_hands[i].add(card)
            self.__show_hands()
        card = self.__deck.draw_card()
        self.__house_hand.add_before(card)
        self.__show_hands()
    
    def __check_naturals(self):
        self.__results = [None] * self.__num_starting_hands
        if self.__house_hand.is_21:
            self.__house_hand[1].flip()
            self.__show_hands()
            print(f'<House> has a natural', end='\n\n')
        for i in range(len(self.__player_hands)):
            if self.__player_hands[i].is_21 and not self.__house_hand.is_21:
                print(f'<Hand {i+1}> has a natural', end='\n\n')
                self.__results[i] = int(round(self.__bets[i] * 1.5))
            elif self.__house_hand.is_21 and not self.__player_hands[i].is_21:
                self.__results[i] = -self.__bets[i]
            elif self.__player_hands[i].is_21 and self.__house_hand.is_21:
                print(f'<Hand {i+1}> has a natural', end='\n\n')
                self.__results[i] = 0
    
    def __player_turn(self):
        i = 0
        while i < len(self.__player_hands):
            if self.__results[i] is not None:
                i += 1
                continue
            while True:
                actions = 'Hit? Stand?'
                choices = ('h', 's')
                if self.one_hit_ace_split and self.__player_hands[i].times_split == 1 and self.__player_hands[i][0] == 1:
                    choice = 'h'
                    one_hit_ace = True
                else:
                    one_hit_ace = False
                    if self.money_left - sum(self.__bets) >= self.__bets[i] and (self.can_dd_after_split or self.__player_hands[i].times_split == 0):
                        actions += ' Double Down?'
                        choices += ('dd',)
                    if (self.__player_hands[i].can_split if self.can_split_diff_tens else self.__player_hands[i].can_split_same) and self.__player_hands[i].times_split < self.resplit_to and self.money_left - sum(self.__bets) >= self.__bets[i]:
                        actions += ' Split?'
                        choices += ('sp',)
                    choice = input(f'<Hand {i+1}> ({self.__player_hands[i].display_score}): {actions} ({"/".join(choices)}): ').lower()
                    assert choice in choices, f'choice must be one of ({"/".join(choices)})'
                    print()
                if choice == 's':
                    self.__show_hands()
                    break
                if choice == 'dd':
                    self.__bets[i] *= 2
                if choice == 'sp':
                    hand = self.__player_hands[i].split()
                    self.__player_hands.insert(i+1, hand)
                    self.__bets.insert(i+1, self.__bets[i])
                    self.__results.insert(i+1, None)
                    self.__show_hands()
                    continue
                card = self.__deck.draw_card()
                self.__player_hands[i].add(card)
                self.__show_hands()
                if self.__player_hands[i].is_busted:
                    print(f'<Hand {i+1}> is busted with {self.__player_hands[i].score}', end='\n\n')
                    self.__results[i] = -self.__bets[i]
                    break
                if self.__player_hands[i].is_21:
                    if self.natural_after_split and len(self.__player_hands[i]) == 2:
                        print(f'<Hand {i+1}> has a natural', end='\n\n')
                        self.__results[i] = int(round(self.__bets[i] * 1.5))
                    else:
                        print(f'<Hand {i+1}> has 21', end='\n\n')
                    break
                if choice == 'dd' or one_hit_ace:
                    break
            i += 1
    
    def __house_turn(self):
        if any(result is None for result in self.__results):
            self.__house_hand[1].flip()
            self.__show_hands()
            while self.__house_hand.score < 17 or self.__house_hand.is_s17 and self.h17:
                card = self.__deck.draw_card()
                self.__house_hand.add(card)
                self.__show_hands()
            if self.__house_hand.is_busted:
                print(f'<House> is busted with {self.__house_hand.score}', end='\n\n')
                for i in range(len(self.__player_hands)):
                    if self.__results[i] is None:
                        self.__results[i] = self.__bets[i]
    
    def __show_results(self):
        if not self.__house_hand.has_flipped_cards and not (self.__house_hand.score == 21 and len(self.__house_hand) == 2):
            print(f'<House> has {self.__house_hand.score}', end='\n\n')
        for i in range(len(self.__player_hands)):
            if self.__house_hand.score == 21 and len(self.__house_hand) == 2:
                if self.__player_hands[i].score == 21:
                    print(f'<Hand {i+1}> ties with a natural -> +$0')
                else:
                    print(f'<Hand {i+1}> loses with no natural -> -${self.__bets[i]}')
            elif self.__player_hands[i].score == 21 and self.__results[i] == round(self.__bets[i] * 1.5):
                print(f'<Hand {i+1}> wins with a natural -> +${self.__results[i]}')
            elif self.__player_hands[i].score > 21:
                print(f'<Hand {i+1}> loses with a busted {self.__player_hands[i].score} -> -${self.__bets[i]}')
            elif self.__house_hand.is_busted:
                print(f'<Hand {i+1}> wins with a non-busted {self.__player_hands[i].score} -> +${self.__bets[i]}')
            elif self.__player_hands[i].score < self.__house_hand.score:
                print(f'<Hand {i+1}> loses with {self.__player_hands[i].score} < {self.__house_hand.score} -> -${self.__bets[i]}')
                self.__results[i] = -self.__bets[i]
            elif self.__player_hands[i].score > self.__house_hand.score:
                print(f'<Hand {i+1}> wins with {self.__player_hands[i].score} > {self.__house_hand.score} -> +${self.__bets[i]}')
                self.__results[i] = self.__bets[i]
            else:
                print(f'<Hand {i+1}> ties with {self.__player_hands[i].score} = {self.__house_hand.score} -> +$0')
                self.__results[i] = 0
        print()
    
    def __end_round(self):
        result = sum(self.__results)
        self.__money_left += result
        if result < 0:
            neg = True
            result = abs(result)
        else:
            neg = False
        print(f'Result: {"-" if neg else "+"}${result} -> ${self.money_left}', end='\n\n')
        self.__rounds_played += 1
        if self.money_left >= self.min_bet:
            play_again = input('Play Again? (y/n): ').lower()
            assert play_again in ('y', 'n'), 'answer must be y or n'
            if play_again == 'y':
                self.play_round()
            else:
                clear_output()
        else:
            clear_output()

    def play_round(self) -> None:
        self.__start_round()
        self.__collect_bets()
        self.__deal()
        self.__check_naturals()
        self.__player_turn()
        self.__house_turn()
        self.__show_results()
        self.__end_round()
    
    @property
    def deck(self) -> Deck:
        return self.__deck

    @property
    def money_left(self) -> int:
        return self.__money_left
    
    @property
    def h17(self) -> bool:
        return self.__h17

    @property
    def can_dd_after_split(self) -> bool:
        return self.__can_dd_after_split
    
    @property
    def resplit_to(self) -> int:
        return self.__resplit_to

    @property
    def natural_after_split(self) -> bool:
        return self.__natural_after_split
    
    @property
    def can_split_diff_tens(self) -> bool:
        return self.__can_split_diff_tens
    
    @property
    def one_hit_ace_split(self) -> bool:
        return self.__one_hit_ace_split
    
    @property
    def min_bet(self) -> int:
        return self.__min_bet
    
    @property
    def max_bet(self) -> int:
        return self.__max_bet
    
    @property
    def rounds_played(self) -> int:
        return self.__rounds_played
    
    @property
    def num_decks(self) -> int:
        return self.__deck.num_decks
    
    @property
    def has_stop_card(self) -> bool:
        return self.__deck.has_stop_card
    
    @property
    def num_cards(self) -> int:
        return self.__deck.num_cards

In [63]:
blkj = Blackjack()

In [64]:
blkj.play_round()